This code is part of a preprocessing step for static layers used in firenet. It involves loading various landfire layers, reprojecting them to a common coordinate reference system (CRS), and then stacking them together into a single multi-layered dataset. Landfire data gives spatial information such as elevation, vegetation height etc. that are useful to feed into firenet when it makes inference via interpreting spectral satellite data.

The reprojected layers are matched to the CRS of a median image to ensure spatial alignment across all layers. This combined dataset is then saved as a NetCDF file for further use. 

Memory management is also considered by manually invoking garbage collection after processing each layer to ensure the environment won't crash.


In [1]:
import gc
import xarray as xr
import rioxarray

# Load the landfire layers
landfire_tif_paths = ['/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVH_220_CONUS/Tif/LC22_EVH_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FBFM40_220_CONUS/Tif/LC22_F40_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FVH_220_CONUS/Tif/LC22_FVH_220.tif']

# Initialize an empty dictionary to store the datasets
datasets = {}

In [2]:
import rioxarray
from rasterio.enums import Resampling
import os

# Define the target resolution
# Create an empty dictionary to hold the reprojected/resampled datasets
landfire_datasets = {}

median_ds = rioxarray.open_rasterio('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/reprojected_median_ds.nc')

In [3]:
median_ds.rio.crs

CRS.from_epsg(5070)

In [5]:
# Loop through each Landfire TIFF file path
for tif_path in landfire_tif_paths[0:]:
    print(tif_path)
    # Open the TIFF file as an xarray dataset
    landfire_ds = rioxarray.open_rasterio(tif_path, chunks={'x': 1024, 'y': 1024})
    
    # Reproject and resample the dataset to the target resolution
    # Assuming the median_ds has the target CRS and resolution
    landfire_reprojected = landfire_ds.rio.reproject_match(median_ds)
    
    del landfire_ds
    gc.collect()  # Invoke garbage collection

    # Get the variable name from the TIFF file name
    var_name = os.path.splitext(os.path.basename(tif_path))[0]
    
    # Define the new file name
    new_file_name = f"/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/{var_name}_reprojected.tif"
    # Save the reprojected/resampled dataset as a new TIFF file
    landfire_reprojected.rio.to_raster(new_file_name)
    
    # Close the dataset
    landfire_reprojected.close()
    del landfire_reprojected
    gc.collect()  # Invoke garbage collection

/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVH_220_CONUS/Tif/LC22_EVH_220.tif
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FBFM40_220_CONUS/Tif/LC22_F40_220.tif
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FVH_220_CONUS/Tif/LC22_FVH_220.tif


In [7]:
import glob
import rioxarray
import os

# Get a list of all reprojected TIFF files
reprojected_tif_paths = glob.glob('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/*_reprojected.tif')

# Initialize an empty dictionary to store the datasets
datasets = {}

# Loop through each reprojected TIFF file path
for tif_path in reprojected_tif_paths:
    # Open the TIFF file as an xarray dataset
    ds = rioxarray.open_rasterio(tif_path)
    
    # Get the variable name from the TIFF file name
    var_name = os.path.splitext(os.path.basename(tif_path))[0].replace('_reprojected', '')
    
    # Add the dataset to the dictionary, using the variable name as the key
    datasets[var_name] = ds

# Combine all datasets into a single xarray dataset
combined_ds = xr.Dataset(datasets)

# Save the combined dataset as a NetCDF file
combined_ds.to_netcdf('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/combined_landfire.nc')

In [8]:
combined_ds



<xarray.Dataset>
Dimensions:        (band: 1, x: 3890, y: 3247)
Coordinates:
  * band           (band) int64 1
  * x              (x) float64 -2.643e+06 -2.642e+06 ... -1.185e+06 -1.184e+06
  * y              (y) float64 2.927e+06 2.926e+06 ... 1.71e+06 1.709e+06
    spatial_ref    int64 0
Data variables:
    LC22_CBD_220   (band, y, x) int16 ...
    LC22_EVH_220   (band, y, x) int16 ...
    LC20_Elev_220  (band, y, x) int16 ...
    LC22_EVC_220   (band, y, x) int16 ...
    LC22_FVH_220   (band, y, x) int16 ...
    LC22_F40_220   (band, y, x) int16 ...